In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import time
import logging
import requests
import pandas as pd
from kafka import KafkaProducer
import json
import psycopg2
from sqlalchemy import create_engine
from scipy.special import boxcox
def zscore_normalization(df, name):
    mean = df[name].mean()
    sd = df[name].std()

    df[name] = (df[name] - mean) / sd
def preprocess(df):
    df = df.drop(columns=['Name', 'md5'])
    for i in df.columns:
        if i != 'legitimate':
            df[i] = boxcox(df[i], 0.5)
            zscore_normalization(df, i)
    correlation_matrix = df.corr()
    cols_to_drop = []
    for i in df.columns:
        for j in df.columns:
            if i != j and i != 'legitimate' and j != 'legitimate' and abs(correlation_matrix[i][j]) > 0.6 and i not in cols_to_drop and j not in cols_to_drop:
                cols_to_drop.append(i)
    cols_to_drop = set(cols_to_drop)
    df.drop(columns=cols_to_drop, inplace=True)
    return df

# Load data
file_path = "https://raw.githubusercontent.com/tsimhadri-ews/internproject/malware-detection-0/src/MalwareData.csv"
df = pd.read_csv(file_path,sep='|')
df = preprocess(df)
df = df.drop(columns=['legitimate'])
# Convert column names to lowercase
df.columns = [col.lower() for col in df.columns]

# Create an empty DataFrame with the same columns to create the table
columns_df = pd.DataFrame(columns=df.columns)
print(f"Total records after processing: {len(columns_df)}")

# PostgreSQL database connection details
db_details = {
    'dbname': 'mydb',
    'user': 'exampleuser',
    'password': 'abc',
    'host': '10.100.107.105',
    'port': 5432
}

# Connection string
conn_string = f"postgresql+psycopg2://{db_details['user']}:{db_details['password']}@{db_details['host']}:{db_details['port']}/{db_details['dbname']}"

# Create an engine with increased timeout settings
engine = create_engine(conn_string, connect_args={"connect_timeout": 10}, execution_options={"timeout": 300})

start_time = time.time()

# Create the table with only column names
table_name = 'malware_data'

try:
    # Use `if_exists='replace'` to create a new table structure or replace the existing one
    columns_df.to_sql(table_name, engine, if_exists='replace', index=False)
    print(f"Table '{table_name}' created with only column names successfully.")
except Exception as e:
    print(e)

end_time = time.time()
duration = end_time - start_time

print(f"Time taken to create the table with column names: {duration} seconds")
